In [1]:
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS

ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor(buffered=True)

DB_NAME = "blp_dataset"
TABLE_NAME = "image_hashes"

cursor.execute("USE {}".format(DB_NAME))

In [3]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS image_hashes ("
    "  file TEXT NOT NULL,"
    "  hash_file VARCHAR(255) PRIMARY KEY NOT NULL"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)


table_schema = (
    "CREATE TABLE IF NOT EXISTS repeated_image_hashes ("
    "  file VARCHAR(255) PRIMARY KEY NOT NULL,"
    "  hash_file TEXT NOT NULL"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [4]:
## CLEAR REPEATED IMAGES

# == Files config ==
MAIN_FOLDER = r"D:\images\excluded_product"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")


    img_hash = imagehash.phash(Image.open(image_path))
    image_name = os.path.splitext(image)[0]

    cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
    has_hash = cursor.fetchone()
    
    if has_hash[0] == 0:
        query = "INSERT INTO image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, str(img_hash))

        cursor.execute(query, values)
        conn.commit()
    else:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, str(img_hash))

        cursor.execute(query, values)
        conn.commit()

In [5]:
# REMOVE IMAGES IN REPEATED TABLE
cursor.execute("SELECT * FROM repeated_image_hashes")
repeated = cursor.fetchall()

MAIN_FOLDER = r"D:\images\excluded_product"
folder = Path(MAIN_FOLDER)

for res in repeated:
    image_path = Path(f"{folder}\{res[0]}.jpeg")
    os.remove(image_path)

In [ ]:
## CLEAR REPEATED IMAGES IN DIFFERENT FOLDERS

# == Files config ==
MAIN_FOLDER = r"D:\images\_haven"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")


    img_hash = imagehash.phash(Image.open(image_path))
    image_name = os.path.splitext(image)[0]

    cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
    has_hash = cursor.fetchone()
    
    if has_hash[0] == 0:
        print("Safe")
    else:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, str(img_hash))

        cursor.execute(query, values)
        conn.commit()

In [2]:
# Get list of numbers in sequence that don't have files

count = 0
nums = []
MAIN_FOLDER = r"D:\images\illegal"

while (len(nums) < 169 or count < 48771):
    img = Path(f"{MAIN_FOLDER}\{count}.jpeg")
    if not img.exists():
        nums.append(count)

    count = count + 1


NameError: name 'Path' is not defined

In [11]:
# Rename images based on list of numbers that don't exist into illegal folder

folder = Path("D:\images\excluded_product")
cont = 1

for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")

    try:
        img = Image.open(image_path).convert('RGB').save(f'{folder}\{nums[cont]}.jpeg')
        cont = cont + 1
    except:
        print(image_name)